# Drone Simulation with PyBullet

This notebook simulates a quadrotor drone using PyBullet and plots the angular velocity over time.

In [1]:
import pybullet as p
import pybullet_data
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib notebook

pybullet build time: Sep  3 2024 12:59:12


## Constants and Parameters

In [2]:
# ----- Constants and Parameters ----- #
pwm_min = 1000
pwm_max = 2000
previous_motor_outputs = np.zeros(4)  # Store previous motor outputs
motor_time_constant = 0.3  # Time constant from your transfer function (0.3s)


def pwm_to_thrust(pwm):
    """Convert PWM to thrust in Newtons (using corrected quadratic model)."""
    return 2.37e-5 * pwm ** 2 - 4.71e-2 * pwm + 23.58

# Drone physical parameters
arm_length = 0.1  # meters
mass = 1.0  # kg
gravity = -9.81

# Motor layout and directions (X configuration)
motor_dirs = [
    np.array([arm_length, arm_length, 0]),  # Front Right (CCW)
    np.array([-arm_length, arm_length, 0]),  # Front Left  (CW)
    np.array([-arm_length, -arm_length, 0]),  # Rear Left   (CW)
    np.array([arm_length, -arm_length, 0]),  # Rear Right  (CCW)
]

## Initialize PyBullet

In [3]:
# ----- Initialize PyBullet ----- #
p.connect(p.GUI)
p.setGravity(0, 0, gravity)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.loadURDF("plane.urdf")
start_pos = [0, 0, 10]
start_orientation = p.getQuaternionFromEuler([0, 0, 0])
drone = p.loadURDF("quadrotor.urdf", start_pos, start_orientation, useFixedBase=False)

# Optimize physics for speed
p.setPhysicsEngineParameter(numSolverIterations=5)
p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
p.configureDebugVisualizer(p.COV_ENABLE_SHADOWS, 0)

## PID Controller

In [4]:
# ----- PID Controller ----- #
kp, ki, kd = 3, 0.0, 0.00  # Adjusted gains
integral_error = np.zeros(3)
prev_error = np.zeros(3)


def pid_controller(w_desired, w_actual, dt):
    global integral_error, prev_error
    error = w_desired - w_actual
    integral_error += error * dt
    integral_error = np.clip(integral_error, -1, 1)
    derivative = (error - prev_error) / dt
    prev_error = error
    return -(kp * error + ki * integral_error + kd * derivative)

## Main Simulation Loop

In [5]:
# ----- Main Simulation Loop ----- #
dt = 1. / 833.
w_desired = np.array([0.0, 1.5, 0.0])  # Only roll for testing

# For debugging
debug_lines = [None, None, None, None]
debug_text = p.addUserDebugText("", [0, 0, 0], textColorRGB=[1, 1, 1])
hover_pwm = 1600  # Base hover PWM

# Initialize motors at hover PWM
previous_motor_outputs = np.ones(4) * hover_pwm

# Control scaling factors
roll_scale = 1.0
pitch_scale = 1.0
yaw_scale = 1.0

# Speed optimization parameters
vis_update_freq = 20  # Update visualization every N iterations
print_freq = 50  # Print debug info every N iterations
steps_per_iter = 2  # Physics steps per iteration
real_time_factor = 4.0  # Run faster than real-time

# Data collection for plotting
max_iterations = 2000  # Set max iterations to limit simulation time
time_points = []
roll_velocity = []
pitch_velocity = []
yaw_velocity = []
desired_roll = []
desired_pitch = []
desired_yaw = []

In [6]:


for iteration in range(max_iterations):
    # Record time point
    time_points.append(iteration * dt)

    # Get orientation and angular velocity
    pos, ori = p.getBasePositionAndOrientation(drone)
    linear_vel, ang_vel_world = p.getBaseVelocity(drone)

    # Convert to body frame
    rot_mat = np.array(p.getMatrixFromQuaternion(ori)).reshape(3, 3)
    ang_vel_body = rot_mat.T @ np.array(ang_vel_world)
    ang_vel = np.array([ang_vel_body[1], ang_vel_body[0], ang_vel_body[2]])

    # Record angular velocity for plotting
    roll_velocity.append(ang_vel[0])
    pitch_velocity.append(ang_vel[1])
    yaw_velocity.append(ang_vel[2])
    desired_roll.append(w_desired[0])
    desired_pitch.append(w_desired[1])
    desired_yaw.append(w_desired[2])

    # Update debug text (less frequently)
    if iteration % vis_update_freq == 0:
        text = f"Ang Vel: [{ang_vel[0]:.2f}, {ang_vel[1]:.2f}, {ang_vel[2]:.2f}]"
        p.removeUserDebugItem(debug_text)
        debug_text = p.addUserDebugText(text, [pos[0], pos[1], pos[2] + 0.3], textColorRGB=[1, 1, 1])

    # Run PID controller to get control output
    control_output = pid_controller(w_desired, ang_vel, dt)
    roll_cmd, pitch_cmd, yaw_cmd = control_output

    # Calculate motor PWMs (vectorized approach)
    motor_pwms = np.ones(4) * hover_pwm

    # Apply roll, pitch, yaw commands
    roll_effect = np.array([-1, 1, 1, -1]) * roll_cmd * roll_scale
    pitch_effect = np.array([-1, -1, 1, 1]) * pitch_cmd * pitch_scale
    yaw_effect = np.array([1, -1, -1, 1]) * yaw_cmd * yaw_scale

    motor_pwms += roll_effect + pitch_effect + yaw_effect
    motor_pwms = np.clip(motor_pwms, pwm_min, pwm_max)

    # Apply first-order filter (vectorized)
    alpha = dt / (motor_time_constant + dt)
    filtered_pwms = (1 - alpha) * previous_motor_outputs + alpha * motor_pwms
    previous_motor_outputs = filtered_pwms.copy()

    # Calculate thrusts (vectorized)
    actual_thrusts = 2.37e-5 * filtered_pwms ** 2 - 4.71e-2 * filtered_pwms + 23.58

    # Print debug info (less frequently)
    if iteration % print_freq == 0:
        print(f"Iteration {iteration}")
        print(f"Angular Velocity: {ang_vel}")
        print(f"Control Output: [{roll_cmd:.4f}, {pitch_cmd:.4f}, {yaw_cmd:.4f}]")
        print(f"Motor PWMs: {motor_pwms}")
        print(f"Thrusts: {actual_thrusts}")
        print("-" * 50)

    # Apply forces at motor positions
    for i, thrust in enumerate(actual_thrusts):
        pos_world = motor_dirs[i]
        force = [0, 0, thrust]
        p.applyExternalForce(objectUniqueId=drone,
                             linkIndex=-1,
                             forceObj=force,
                             posObj=pos_world,
                             flags=p.LINK_FRAME)

    # Visualize thrust as debug lines (less frequently)
    if iteration % vis_update_freq == 0:
        for i, thrust in enumerate(actual_thrusts):
            if debug_lines[i] is not None:
                p.removeUserDebugItem(debug_lines[i])
            start = np.array(pos) + p.rotateVector(ori, motor_dirs[i])
            end = start + p.rotateVector(ori, [0, 0, thrust * 0.01])
            debug_lines[i] = p.addUserDebugLine(start, end, [1, 0, 0], 2)
    # Add this inside your simulation loop
    if iteration % vis_update_freq == 0:  # Update camera periodically
        pos, ori = p.getBasePositionAndOrientation(drone)
        # Parameters: camera distance, yaw, pitch, target position
        p.resetDebugVisualizerCamera(
            cameraDistance=2.0,  # Distance from target
            cameraYaw=45.0,      # Camera rotation in horizontal plane
            cameraPitch=-30.0,   # Camera tilt (negative looks downward)
            cameraTargetPosition=pos  # Position to focus on (drone position)
        )
    # Run multiple simulation steps per iteration
    for _ in range(steps_per_iter):
        p.stepSimulation()

    # Sleep less for faster than real-time simulation
    time.sleep(dt / real_time_factor)  # Comment out completely for maximum speed

p.disconnect()

Iteration 0
Angular Velocity: [0. 0. 0.]
Control Output: [-0.0000, -4.5000, -0.0000]
Motor PWMs: [1604.5 1604.5 1595.5 1595.5]
Thrusts: [8.89251547 8.89251547 8.89148454 8.89148454]
--------------------------------------------------
Iteration 50
Angular Velocity: [0.         0.06427073 0.        ]
Control Output: [-0.0000, -4.3072, -0.0000]
Motor PWMs: [1604.3071878 1604.3071878 1595.6928122 1595.6928122]
Thrusts: [8.91547556 8.91547556 8.86855602 8.86855602]
--------------------------------------------------
Iteration 100
Angular Velocity: [0.         0.23203014 0.        ]
Control Output: [-0.0000, -3.8039, -0.0000]
Motor PWMs: [1603.80390958 1603.80390958 1596.19609042 1596.19609042]
Thrusts: [8.93239938 8.93239938 8.85169406 8.85169406]
--------------------------------------------------
Iteration 150
Angular Velocity: [0.         0.46764299 0.        ]
Control Output: [-0.0000, -3.0971, -0.0000]
Motor PWMs: [1603.09707103 1603.09707103 1596.90292897 1596.90292897]
Thrusts: [8.94303

## Plot Results

Now let's visualize the angular velocity data.

In [7]:
# Create plots after simulation is complete
plt.figure(figsize=(12, 8))

# Plot angular velocities
plt.subplot(3, 1, 1)
plt.plot(time_points, roll_velocity, 'r-', label='Roll Velocity')
plt.plot(time_points, desired_roll, 'r--', label='Desired Roll')
plt.ylabel('Roll Rate (rad/s)')
plt.grid(True)
plt.legend()
plt.title('Drone Angular Velocity vs Time')

plt.subplot(3, 1, 2)
plt.plot(time_points, pitch_velocity, 'g-', label='Pitch Velocity')
plt.plot(time_points, desired_pitch, 'g--', label='Desired Pitch')
plt.ylabel('Pitch Rate (rad/s)')
plt.grid(True)
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(time_points, yaw_velocity, 'b-', label='Yaw Velocity')
plt.plot(time_points, desired_yaw, 'b--', label='Desired Yaw')
plt.xlabel('Time (s)')
plt.ylabel('Yaw Rate (rad/s)')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()  # Only display the plot without saving

<IPython.core.display.Javascript object>

## Analysis

Let's analyze the performance of the drone control system by looking at settling time, overshoot, and steady-state error.

In [9]:
# Calculate performance metrics
import numpy as np

# For roll velocity (which is our primary control objective)
roll_data = np.array(roll_velocity)
desired_roll_data = np.array(desired_roll)
time_data = np.array(time_points)

# Calculate error
roll_error = desired_roll_data - roll_data

# Steady state error (average of last 10% of data)
ss_index = int(len(roll_data) * 0.9)
steady_state_error = np.mean(np.abs(roll_error[ss_index:]))

# Maximum error (overshoot or undershoot)
max_error = np.max(np.abs(roll_error))

# Display results
print(f"Steady-state error: {steady_state_error:.4f} rad/s")
print(f"Maximum error: {max_error:.4f} rad/s")

# Plot error
plt.figure(figsize=(10, 5))
plt.plot(time_data, roll_error)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Time (s)')
plt.ylabel('Error (rad/s)')
plt.title('Roll Rate Error vs Time')
plt.grid(True)
plt.show()

Steady-state error: 0.0000 rad/s
Maximum error: 0.0280 rad/s


<IPython.core.display.Javascript object>

In [9]:
# Calculate performance metrics
import numpy as np

# For pitch velocity (our control objective in this case)
pitch_data = np.array(pitch_velocity)
desired_pitch_data = np.array(desired_pitch)
time_data = np.array(time_points)

# Calculate error
pitch_error = desired_pitch_data - pitch_data

# Steady state error (average of last 10% of data)
ss_index = int(len(pitch_data) * 0.9)
steady_state_error = np.mean(np.abs(pitch_error[ss_index:]))

# Maximum error (overshoot or undershoot)
max_error = np.max(np.abs(pitch_error))

# Display results
print(f"Pitch steady-state error: {steady_state_error:.4f} rad/s")
print(f"Pitch maximum error: {max_error:.4f} rad/s")

# Plot error
plt.figure(figsize=(10, 5))
plt.plot(time_data, pitch_error)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Time (s)')
plt.ylabel('Error (rad/s)')
plt.title('Pitch Rate Error vs Time')
plt.grid(True)
plt.show()

Pitch steady-state error: 1.5043 rad/s
Pitch maximum error: 57.8328 rad/s


<IPython.core.display.Javascript object>

## Experiment with Different PID Gains

You can experiment with different PID gains to see how they affect the drone's performance. Uncomment and run the cell below to see the impact of different Kp values.

In [ ]:
# # This cell is optional - run it to compare different Kp values
# import matplotlib.pyplot as plt
# import numpy as np
# 
# # Define a simple system model for quick testing
# def simulate_response(kp, ki, kd, setpoint=0.5, duration=2.0, dt=0.01):
#     t = np.arange(0, duration, dt)
#     y = np.zeros_like(t)
#     e_prev = 0
#     e_integral = 0
#     
#     # Simple second-order system
#     for i in range(1, len(t)):
#         e = setpoint - y[i-1]
#         e_integral += e * dt
#         e_derivative = (e - e_prev) / dt
#         u = kp * e + ki * e_integral + kd * e_derivative
#         
#         # System dynamics (quadrotor-like)
#         y[i] = y[i-1] + dt * (2 * u - 0.5 * y[i-1])
#         e_prev = e
#     
#     return t, y
# 
# # Test different Kp values
# kp_values = [1.0, 3.0, 5.0, 10.0]
# plt.figure(figsize=(10, 6))
# 
# for kp in kp_values:
#     t, y = simulate_response(kp=kp, ki=0.1, kd=0.05)
#     plt.plot(t, y, label=f'Kp = {kp}')
# 
# plt.axhline(y=0.5, color='r', linestyle='--', label='Setpoint')
# plt.xlabel('Time (s)')
# plt.ylabel('Angular Velocity (rad/s)')
# plt.title('Effect of Different Kp Values on Step Response')
# plt.legend()
# plt.grid(True)
# plt.show()